In [1]:
import sys
sys.path.append("../")
from initial_dataSet import DataSet
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
basedir = '../'
baseline_list=('DINA','NCD')
baseline=baseline_list[1]

dataSet_list = ('ASSIST_0910', 'ASSIST_2017','JUNYI', 'MathEC','KDDCUP')
save_list = ('a0910/', 'a2017/', 'junyi/', 'math_ec/', 'kddcup/')

dataSet_idx=0
data_set_name = dataSet_list[dataSet_idx]
dataSet = DataSet(basedir, data_set_name)

data_dir='E:/PY_Project/知识点交互CDM/Experiment/output/'+save_list[dataSet_idx]

train=pd.read_csv(data_dir+'train.csv')
test=pd.read_csv(data_dir+'test.csv')

DataSet: ASSIST_0910


In [3]:
def get_item_concept_df(item) -> pd.DataFrame:
    item = item[~item.index.duplicated()]  # 去除重复项
    item_list, concept_list = [], []
    for idx in item.index:
        now_concept_list = eval(item.loc[idx, 'knowledge_code'])
        item_list.extend([idx] * len(now_concept_list))
        concept_list.extend(now_concept_list)
    return pd.DataFrame({'item': item_list, 'concept': concept_list}).astype('int')

In [4]:
def get_DOA(record,cogn_state,item_conc):
    doa_list=[]
    unique_item=record['item_id'].unique()
    for k in tqdm(range(1,cogn_state.shape[1]+1)):
        cong_doa_count=0
        item_cong_doa_count=0
        item_k=item_conc[item_conc['concept']==k-1]['item'].unique()
        item_list=list(set(unique_item)&set(item_k))
        record_k=record.set_index('item_id').loc[item_list,:]
        if len(item_list)>0 and len(record_k)>0:
            for j in item_list:
                record_j=record_k.loc[j,:]
                stu_j=np.array(record_j['user_id']).reshape(-1).astype('int')
                cong_j=cogn_state[stu_j-1,k-1]
                sort_stu_idx=cong_j.argsort() # 从小到大
                sort_score_j=np.array(record_j['score']).reshape(-1)[sort_stu_idx]
                sort_cong_j=cong_j[sort_stu_idx]
                for i in range(len(sort_stu_idx)):
                    fliter=((sort_cong_j[i:]-sort_cong_j[i])>=0)
                    cong_doa_count+=fliter.sum()
                    score_i=sort_score_j[i:][fliter]
                    if len(score_i)>0:
                        item_cong_doa_count+=((score_i-sort_score_j[i])>=0).sum()
            if cong_doa_count>0:
                doa_list.append(item_cong_doa_count/cong_doa_count)
    return np.mean(doa_list)

In [5]:
read_dir='./'+baseline+'/output/'+save_list[dataSet_idx]

cogn_state=np.loadtxt(read_dir+'cognitive_state.csv',delimiter=',')
item_conc=get_item_concept_df(dataSet.item)

In [6]:
train_doa=get_DOA(train,cogn_state,item_conc)
print('{} 的训练集DOA = {}'.format(data_set_name,train_doa))

100%|██████████| 110/110 [00:08<00:00, 12.89it/s]

ASSIST_0910 的训练集DOA = 0.883727440557463


In [7]:
test_doa=get_DOA(test,cogn_state,item_conc)
print('{} 的测试集DOA = {}'.format(data_set_name,test_doa))

100%|██████████| 110/110 [00:08<00:00, 12.88it/s]

ASSIST_0910 的测试集DOA = 0.8801513707671896
